In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pulp import *
import pandas as pd
import os, glob
import seaborn as sns
from scipy.stats import kruskal
import scikit_posthocs as sp
from scipy.stats import mannwhitneyu

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='/Users/fa/.config/gcloud/application_default_credentials.json' 
os.environ['GCLOUD_PROJECT'] = 'som-nero-phi-jonc101' 
%load_ext google.cloud.bigquery

from google.cloud import bigquery
client=bigquery.Client()
from google.cloud import bigquery_storage_v1

## Read patients_Notes_EHR

In [3]:
%%bigquery Train_set_df
select * from `som-nero-phi-jonc101.blood_culture_stewardship.cohort` where order_year>=2015 and order_year<2022

Could not save output to variable 'Train_set_df'.

ERROR:
 Timeout of 600.0s exceeded, last exception: HTTPSConnectionPool(host='oauth2.googleapis.com', port=443): Max retries exceeded with url: /token (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x142be0290>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))


In [4]:
%%bigquery Val_set_df
select * from `som-nero-phi-jonc101.blood_culture_stewardship.cohort` where order_year>=2022 and order_year<2023

Query is running:   0%|          |

Downloading:   0%|          |

In [5]:
%%bigquery Test_set_df
select * from `som-nero-phi-jonc101.blood_culture_stewardship.cohort` where order_year>=2023 

Query is running:   0%|          |

Downloading:   0%|          |

In [6]:
%%bigquery Notes_df
select * from `som-nero-phi-jonc101.blood_culture_stewardship.EDProviderNotes`

Query is running:   0%|          |

Downloading:   0%|          |

In [7]:
Train_set_df_note=pd.merge(Train_set_df,Notes_df,how='inner',on=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'])
Train_set_df_note[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

Val_set_df_note=pd.merge(Val_set_df,Notes_df,how='inner',on=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'])
Val_set_df_note[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

Test_set_df_note=pd.merge(Test_set_df,Notes_df,how='inner',on=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'])
Test_set_df_note[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

NameError: name 'Train_set_df' is not defined

In [ ]:
Train_set_df_note=Train_set_df_note[Train_set_df_note.deid_note_text.notnull()]
Train_set_df_note[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

Val_set_df_note=Val_set_df_note[Val_set_df_note.deid_note_text.notnull()]
Val_set_df_note[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

Test_set_df_note=Test_set_df_note[Test_set_df_note.deid_note_text.notnull()]
Test_set_df_note[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

# Approach 1

In [ ]:
Train_set_df_note.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime','notedatetime','deid_note_text'],inplace=True)
Train_set_df_note.shape

Val_set_df_note.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime','notedatetime','deid_note_text'],inplace=True)
Val_set_df_note.shape

Test_set_df_note.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime','notedatetime','deid_note_text'],inplace=True)
Test_set_df_note.shape


In [ ]:
Train_set_df_note['blood_culture_order_datetime'] = pd.to_datetime(Train_set_df_note['blood_culture_order_datetime'])
Train_set_df_note['notedatetime'] = pd.to_datetime(Train_set_df_note['notedatetime'])
Train_set_df_note['time-diff']=(Train_set_df_note['blood_culture_order_datetime']-Train_set_df_note['notedatetime']).dt.total_seconds()/3600
Train_set_df_note['time-diff']=Train_set_df_note['time-diff'].astype(int)
Train_set_df_note=Train_set_df_note[Train_set_df_note['time-diff']>=-24]


Val_set_df_note['blood_culture_order_datetime'] = pd.to_datetime(Val_set_df_note['blood_culture_order_datetime'])
Val_set_df_note['notedatetime'] = pd.to_datetime(Val_set_df_note['notedatetime'])
Val_set_df_note['time-diff']=(Val_set_df_note['blood_culture_order_datetime']-Val_set_df_note['notedatetime']).dt.total_seconds()/3600
Val_set_df_note['time-diff']=Val_set_df_note['time-diff'].astype(int)
Val_set_df_note=Val_set_df_note[Val_set_df_note['time-diff']>=-24]

Test_set_df_note['blood_culture_order_datetime'] = pd.to_datetime(Test_set_df_note['blood_culture_order_datetime'])
Test_set_df_note['notedatetime'] = pd.to_datetime(Test_set_df_note['notedatetime'])
Test_set_df_note['time-diff']=(Test_set_df_note['blood_culture_order_datetime']-Test_set_df_note['notedatetime']).dt.total_seconds()/3600
Test_set_df_note['time-diff']=Test_set_df_note['time-diff'].astype(int)
Test_set_df_note=Test_set_df_note[Test_set_df_note['time-diff']>=-24]


In [ ]:
Train_set_df_note['time-diff']=(Train_set_df_note['blood_culture_order_datetime']-Train_set_df_note['notedatetime']).dt.total_seconds().abs()
Val_set_df_note['time-diff']=(Val_set_df_note['blood_culture_order_datetime']-Val_set_df_note['notedatetime']).dt.total_seconds().abs()
Test_set_df_note['time-diff']=(Test_set_df_note['blood_culture_order_datetime']-Test_set_df_note['notedatetime']).dt.total_seconds().abs()


Train_set_df_note = Train_set_df_note.sort_values('time-diff').drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded', 'blood_culture_order_datetime'], keep='first')
Val_set_df_note = Val_set_df_note.sort_values('time-diff').drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded', 'blood_culture_order_datetime'], keep='first')
Test_set_df_note = Test_set_df_note.sort_values('time-diff').drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded', 'blood_culture_order_datetime'], keep='first')


In [ ]:
Train_set_df_note['doc_id'] = Train_set_df_note.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime']).ngroup() + 1
Val_set_df_note['doc_id'] = Val_set_df_note.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime']).ngroup() + 1
Test_set_df_note['doc_id'] = Test_set_df_note.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime']).ngroup() + 1

In [ ]:
## Evalute GPT4 on whole Notes 

## Evalute GPT4 on whole Notes + EHR ********* (running from this note book)

In [13]:
Test_set_df_note['Label']=(Test_set_df_note['positive_blood_culture']| Test_set_df_note['positive_blood_culture_in_week'])

In [16]:
EHR=[col for col in Test_set_df_note.columns.values if 'min' in col]
EHR.extend([col for col in Test_set_df_note.columns.values if 'max' in col])
EHR.extend([col for col in Test_set_df_note.columns.values if 'avg' in col])
EHR.extend([col for col in Test_set_df_note.columns.values if 'median' in col])
EHR

['min_heartrate',
 'min_resprate',
 'min_temp',
 'min_sysbp',
 'min_diasbp',
 'min_wbc',
 'min_neutrophils',
 'min_lymphocytes',
 'min_hgb',
 'min_plt',
 'min_na',
 'min_hco3',
 'min_bun',
 'min_cr',
 'min_lactate',
 'min_procalcitonin',
 'max_heartrate',
 'max_resprate',
 'max_temp',
 'max_sysbp',
 'max_diasbp',
 'max_wbc',
 'max_neutrophils',
 'max_lymphocytes',
 'max_hgb',
 'max_plt',
 'max_na',
 'max_hco3',
 'max_bun',
 'max_cr',
 'max_lactate',
 'max_procalcitonin',
 'avg_heartrate',
 'avg_resprate',
 'avg_temp',
 'avg_sysbp',
 'avg_diasbp',
 'avg_wbc',
 'avg_neutrophils',
 'avg_lymphocytes',
 'avg_hgb',
 'avg_plt',
 'avg_na',
 'avg_hco3',
 'avg_bun',
 'avg_cr',
 'avg_lactate',
 'avg_procalcitonin',
 'median_heartrate',
 'median_resprate',
 'median_temp',
 'median_sysbp',
 'median_diasbp',
 'median_wbc',
 'median_neutrophils',
 'median_lymphocytes',
 'median_hgb',
 'median_plt',
 'median_na',
 'median_hco3',
 'median_bun',
 'median_cr',
 'median_lactate',
 'median_procalcitonin']

In [17]:
def format_structured_features(row, ehr_columns):
    return "\n".join([f"- {col}: {row[col]}" for col in ehr_columns if pd.notnull(row[col])])


def dict_structured_features(row, ehr_columns):
    EHR={}
    for col in ehr_columns:
        if pd.notnull(row[col]):
            EHR[col]=row[col]
    return EHR
    

prompt_records = []

def split_note_text(note_text, max_words=1000, overlap=10,EHR_IncludeInprompt=True):
    words = note_text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i+max_words]
        chunks.append(" ".join(chunk))
        i += max_words - overlap
    return chunks

for i, row in Test_set_df_note.iterrows():
    structured_block = format_structured_features(row, EHR)
    structured_dict = dict_structured_features(row, EHR)
    note_text = row['deid_note_text']
    note_text = re.sub(r'\s+', ' ', note_text.replace('\n', ' ')).strip()
    
    # Split if longer than 3000 words
    if len(note_text.split()) > 1000:
        note_chunks = split_note_text(note_text)
    else:
        note_chunks = [note_text]

    # Generate one prompt per chunk
    for chunk_index, chunk in enumerate(note_chunks):
        if EHR_IncludeInprompt:
            prompt = (
                "You are an infectious disease expert. Based on the patient information below, determine whether this patient's blood culture is likely to be positive.\n"
                "Respond with **only** 'Yes' or 'No'. Reply 'Yes' if a positive blood culture is likely; otherwise, reply 'No'.\n\n"
                "Patient Information:\n"
                f"{structured_block}\n\n"
                "Clinical Note:\n"
                f"{chunk}\n"
            )
        else:
            prompt = (
                "You are an infectious disease expert. Based on the patient information below, determine whether this patient's blood culture is likely to be positive.\n"
                "Respond with **only** 'Yes' or 'No'. Reply 'Yes' if a positive blood culture is likely; otherwise, reply 'No'.\n\n"
                "Patient Information:\n"
                "Clinical Note:\n"
                f"{chunk}\n"
            )
            
        prompt_records.append({
            'anon_id': row['anon_id'],
            'pat_enc_csn_id_coded': row['pat_enc_csn_id_coded'],
            'order_proc_id_coded': row['order_proc_id_coded'],
            'blood_culture_order_datetime':row['blood_culture_order_datetime'],
            'Label':row['Label'],
            'doc_id':row['doc_id'],
            'chunk_index': chunk_index,
            'prompt': prompt,
            'EHR':[structured_dict]
        })

# Convert to DataFrame
Test_set_prompt_df = pd.DataFrame(prompt_records)

In [18]:
Test_set_prompt_df

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime,Label,doc_id,chunk_index,prompt
0,JC1294554,131355576388,880528649,2023-06-16 03:51:00+00:00,0,1871,0,You are an infectious disease expert. Based on...
1,JC6304492,131355492092,880254774,2023-05-11 05:17:00+00:00,0,18077,0,You are an infectious disease expert. Based on...
2,JC6304492,131355492092,880254774,2023-05-11 05:17:00+00:00,0,18077,1,You are an infectious disease expert. Based on...
3,JC1376167,131346217402,842199916,2023-01-19 00:14:00+00:00,0,2378,0,You are an infectious disease expert. Based on...
4,JC1376167,131346217402,842199916,2023-01-19 00:14:00+00:00,0,2378,1,You are an infectious disease expert. Based on...
...,...,...,...,...,...,...,...,...
55822,JC684427,131366712984,921105215,2023-12-10 21:57:00+00:00,0,22681,1,You are an infectious disease expert. Based on...
55823,JC684427,131366712984,921105215,2023-12-10 21:57:00+00:00,0,22681,2,You are an infectious disease expert. Based on...
55824,JC684427,131366712984,921105213,2023-12-10 21:57:00+00:00,0,22680,0,You are an infectious disease expert. Based on...
55825,JC684427,131366712984,921105213,2023-12-10 21:57:00+00:00,0,22680,1,You are an infectious disease expert. Based on...


In [19]:
import requests
import json
my_key ='5f57674c921c459aa144c7e35360a735'
headers = {'Ocp-Apim-Subscription-Key': my_key, 'Content-Type': 'application/json'}

url = "https://apim.stanfordhealthcare.org/openai-eastus2/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview" 


In [3]:
import pdb
import json
import requests
import pdb
from time import sleep

def gpt4response(row, model="gpt-4",EHR_IncludeInprompt=True):
    prompt=row['prompt']
    ehr_data = row["EHR"]
    doc_id=row['doc_id']
    ehr_data=row['EHR']
    if not prompt:
        return None

    if EHR_IncludeInprompt:
        payload = json.dumps({
            "model": model,
            "messages": [{"role": "user", "content": prompt}
        ]

        })
    else:
        payload = json.dumps({
        "model": model,
        "messages": [
            {"role": "user", "content": prompt},
            {"role": "system", "content": f"Here is the structured EHR data in JSON format:\n{json.dumps(ehr_data)}"}
        ]
    })

    while True:
        try:
            response = requests.post(url, headers=headers, data=payload)
            response_json = response.json()
            if 'choices' in response_json:
                return response_json['choices'][0]['message']['content']
            else:
                print(f"doc_id:{doc_id}")
                print("Response missing 'choices'. Retrying in 10 seconds...")
                sleep(10)
        except Exception as e:
            print(f"Error: {e}. Retrying in 10 seconds...")
            sleep(10)

/Users/ghazal/Documents/StanfordProject/BloodCultureShortage/env_name/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [26]:
Test_set_prompt_df.sort_values(by=['doc_id'],inplace=True)
Test_set_prompt_df = Test_set_prompt_df.reset_index(drop=True)
Test_set_prompt_df

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime,Label,doc_id,chunk_index,prompt
0,JC1000129,131354606122,876959985,2023-04-23 22:44:00+00:00,1,1,2,You are an infectious disease expert. Based on...
1,JC1000129,131354606122,876959985,2023-04-23 22:44:00+00:00,1,1,1,You are an infectious disease expert. Based on...
2,JC1000129,131354606122,876959985,2023-04-23 22:44:00+00:00,1,1,0,You are an infectious disease expert. Based on...
3,JC1000129,131354606122,876959986,2023-04-23 22:44:00+00:00,1,2,2,You are an infectious disease expert. Based on...
4,JC1000129,131354606122,876959986,2023-04-23 22:44:00+00:00,1,2,1,You are an infectious disease expert. Based on...
...,...,...,...,...,...,...,...,...
55822,JC999190,131368104425,926213332,2024-01-04 03:22:00+00:00,1,25137,2,You are an infectious disease expert. Based on...
55823,JC999190,131368104425,926213332,2024-01-04 03:22:00+00:00,1,25137,0,You are an infectious disease expert. Based on...
55824,JC999190,131368104425,926213333,2024-01-04 03:22:00+00:00,1,25138,0,You are an infectious disease expert. Based on...
55825,JC999190,131368104425,926213333,2024-01-04 03:22:00+00:00,1,25138,1,You are an infectious disease expert. Based on...


In [44]:
len_batches=100
n_batches=int(Test_set_prompt_df.shape[0]/len_batches)
for i in range(n_batches):
    DF=Test_set_prompt_df.iloc[i*len_batches:i*len_batches+len_batches]
    DF['GPT_response']=DF.apply(lambda row:gpt4response(row),axis=1)
    filename=f"GPT4_Testset_WholeNotes_EHR_part{i}.csv"
    DF.to_csv(filename,index=False)
    print(f'Batch{i} is done')

print(i)

doc_id:17615
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17615
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17621
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17623
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17624
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17624
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17624
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17628
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17629
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17630
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17630
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17630
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17633
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17634
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17635
Response missing 'choices'. Retrying in 10 second

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_85519/2719832236.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['GPT_response']=DF.apply(lambda row:gpt4response(row),axis=1)


Batch399 is done
doc_id:17661
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17663
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17663
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17663
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17666
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17667
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17668
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17668
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17668
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17670
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17671
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17672
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17672
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17672
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17675
Response missing 'choices'. Retr

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_85519/2719832236.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['GPT_response']=DF.apply(lambda row:gpt4response(row),axis=1)


Batch400 is done
doc_id:17703
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17703
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17703
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17707
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17708
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17709
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17709
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17709
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17716
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17716
Response missing 'choices'. Retrying in 10 seconds...
doc_id:17716
Response missing 'choices'. Retrying in 10 seconds...
Error: HTTPSConnectionPool(host='apim.stanfordhealthcare.org', port=443): Max retries exceeded with url: /openai-eastus2/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview (Caused by NewConnectionError('<urllib3.connection.HTTPSC

...

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_85519/2719832236.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['GPT_response']=DF.apply(lambda row:gpt4response(row),axis=1)


Batch555 is done
doc_id:25048
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25051
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25051
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25052
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25055
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25059
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25060
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25060
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25061
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25062
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25064
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25065
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25065
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25066
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25072
Response missing 'choices'. Retr

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_85519/2719832236.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['GPT_response']=DF.apply(lambda row:gpt4response(row),axis=1)


Batch556 is done
doc_id:25085
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25085
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25088
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25088
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25093
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25093
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25093
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25094
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25094
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25099
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25099
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25100
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25100
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25110
Response missing 'choices'. Retrying in 10 seconds...
doc_id:25110
Response missing 'choices'. Retr

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_85519/2719832236.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['GPT_response']=DF.apply(lambda row:gpt4response(row),axis=1)


In [54]:
DF=Test_set_prompt_df.iloc[i*len_batches:]
DF['GPT_response']=DF.apply(lambda row:gpt4response(row),axis=1)
filename=f"GPT4_Testset_WholeNotes_EHR_part{i}.csv"
DF.to_csv(filename,index=False)
print(f'Batch{i} is done')

Error: HTTPSConnectionPool(host='apim.stanfordhealthcare.org', port=443): Max retries exceeded with url: /openai-eastus2/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x148f13410>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known')). Retrying in 10 seconds...
Error: HTTPSConnectionPool(host='apim.stanfordhealthcare.org', port=443): Max retries exceeded with url: /openai-eastus2/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x148e21cd0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known')). Retrying in 10 seconds...
Error: HTTPSConnectionPool(host='apim.stanfordhealthcare.org', port=443): Max retries exceeded with url: /openai-eastus2/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview (Caus

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_85519/531105688.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['GPT_response']=DF.apply(lambda row:gpt4response(row),axis=1)


### Evalute_GPT4 responses

In [56]:
import glob
Res=[]
for file in glob.glob('./*.csv'):
    if 'GPT4_Testset_WholeNotes_EHR_part' in file:
        if not len(Res):
            Res=pd.read_csv(file)
        else:
            Res=pd.concat([Res,pd.read_csv(file)],axis=0)
Res.head()
        
        

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime,Label,doc_id,chunk_index,prompt,GPT_response
0,JC6329254,131363292433,908281865,2023-11-02 23:19:00+00:00,0,18539,1,You are an infectious disease expert. Based on...,Yes
1,JC6329254,131363292433,908281865,2023-11-02 23:19:00+00:00,0,18539,0,You are an infectious disease expert. Based on...,No
2,JC6329521,131364469824,912584062,2023-11-04 02:52:00+00:00,0,18540,1,You are an infectious disease expert. Based on...,No
3,JC6329521,131364469824,912584062,2023-11-04 02:52:00+00:00,0,18540,0,You are an infectious disease expert. Based on...,Yes
4,JC6329521,131364469824,912584063,2023-11-04 02:52:00+00:00,0,18541,1,You are an infectious disease expert. Based on...,No


In [57]:
Res.loc[Res.GPT_response=='No','GPT_response']=0
Res.loc[Res.GPT_response=='No.','GPT_response']=0
Res.loc[Res.GPT_response=='Yes','GPT_response']=1

Res=Res[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','blood_culture_order_datetime','Label','GPT_response']]
Res.drop_duplicates(inplace=True)
test=Res.groupby(by=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','blood_culture_order_datetime','Label'],as_index=False).max()
test.drop_duplicates(inplace=True)
test
test['Label']=test['Label'].astype('int')

In [58]:
from sklearn.metrics import confusion_matrix, f1_score
import pandas as pd

test['Label'] = test['Label'].astype(int)
test['GPT_response'] = test['GPT_response'].astype(int)

y_true = test['Label']
y_pred =test['GPT_response']

# Confusion matrix: [[TN, FP], [FN, TP]]
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

# Metrics
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
ppv         = tp / (tp + fp) if (tp + fp) > 0 else 0
npv         = tn / (tn + fn) if (tn + fn) > 0 else 0
f1          = f1_score(y_true, y_pred)

# Print results
print(f"Sensitivity (Recall): {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")
print(f"PPV (Precision): {ppv:.2f}")
print(f"NPV: {npv:.2f}")
print(f"F1 Score: {f1:.2f}")

Sensitivity (Recall): 0.79
Specificity: 0.62
PPV (Precision): 0.11
NPV: 0.98
F1 Score: 0.19
